## Homework 10: GPUs

## Due Date: May 3, 2021, 04:00pm

#### Firstname Lastname: WenxinZhang

#### E-mail: wz2164@nyu.edu

#### Enter your solutions and submit this notebook

---

**Problem 1 (100p)**


Write two programs which will be able to run in parallel on a GPU, one using Numba/CUDA (50p), one using PyOpenCL (50p).


Each program will:

- draw two random vectors $\vec u$ and $\vec v$ from $[0,1]^N$ where $N = 10^7$;


- calculate and output similarity between $\vec u$ and $\vec v$.




The similarity between two vectors $\vec u$ and $\vec v$ is defined here as a `cosine` value of the angle between them $\measuredangle \left( \vec u, \vec v \right)$. That is, the program returns: 

$$\cos \left( \measuredangle \left( \vec u, \vec v \right) \right).$$


Note that the output is a real value and must belong to $[-1, 1]$.

# Solution

- Use Numba/CUDA to run the program in parallel on a GPU.

- Use PyOpenCL to run the program in parallel on a GPU.

In [1]:
import pyopencl as cl  # Import the OpenCL GPU computing API
import pyopencl.array as pycl_array  # Import PyOpenCL Array 
import math
import numpy as np  # Import Numpy number tools

np.set_printoptions(suppress=True)
context = cl.create_some_context()  # Initialize the Context
queue = cl.CommandQueue(context)  # Instantiate a Queue

n = 10**7

### Create two random pyopencl arrays
a = pycl_array.to_device(queue, np.random.rand(n).astype(np.float32))
b = pycl_array.to_device(queue, np.random.rand(n).astype(np.float32))  
print('vector u: ', a)
print('vector v: ', b)

### Create an empty pyopencl destination array
res_c = pycl_array.empty_like(a)  
res_d = pycl_array.empty_like(a)  
res_e = pycl_array.empty_like(a) 
program = cl.Program(context, """
__kernel void sum(__global const float *a, __global const float *b, __global float *c, __global float *d, __global float *e)
{
  int i = get_global_id(0);
  c[i] = a[i] * b[i];
  d[i] = a[i] * a[i];
  e[i] = b[i] * b[i];
}""").build()  # Create the OpenCL program

### Enqueue the program for execution and store the result in c
program.sum(queue, a.shape, None, a.data, b.data, res_c.data, res_d.data, res_e.data)  
c_sum = sum(res_c.get())
d_sum = sum(res_d.get())
e_sum = sum(res_e.get())
d_sqrt = math.sqrt(float(d_sum))
e_sqrt = math.sqrt(float(e_sum))
cos = c_sum / (d_sqrt*e_sqrt)
print('cos(u,v)=' , cos)

vector u:  [0.8691765  0.5610658  0.79221517 ... 0.744111   0.5121994  0.5118308 ]
vector v:  [0.9604327  0.8918987  0.39234802 ... 0.05697353 0.03944393 0.33194652]
cos(u,v)= 0.7501078227838023
